In [ ]:
print("This is the test for initial commit!")

In [ ]:
import pandas as pd
print("Pandas imported successfully, version: "+pd.__version__)
import statsmodels as sm
print("Statsmodels imported successfully, version: "+sm.__version__)
import plotly.graph_objects as go
print("Plotly function imported succesfully")
import plotly.express as px
print("Plotly express imported succesfully")
from plotly.subplots import make_subplots

import plotly as py

import plotly.figure_factory as ff

import matplotlib.pyplot as plt

import numpy as np

from statsmodels.graphics.gofplots import qqplot
print("Successful")

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

In [ ]:
data_all = pd.read_csv('/Users/mazutislab/Desktop/SynBio/US_Accidents_May19.csv')

In [ ]:
pd.set_option('display.max_columns', 50)
data_all.head()

In [ ]:
#Selecting only needed data in the given dataset
data_cleaned = data_all[["Severity", "Temperature(F)", "Humidity(%)", "Pressure(in)", "Visibility(mi)", "Wind_Speed(mph)", 
                        "Precipitation(in)"]]
data_cleaned.reset_index()
type(data_cleaned)

#Convert Temperature in Fahrenheit to Celsius
def fahr_to_celsius(temp_fahr):
    """Convert Fahrenheit to Celsius and Return Celsius conversion of input"""
    temp_celsius = (temp_fahr - 32) * 5 / 9
    return temp_celsius

data_cleaned["Temperature(\u00b0C)"] = (fahr_to_celsius(data_cleaned["Temperature(F)"])).round(2)
data_cleaned.drop(['Temperature(F)'], inplace = True, axis = 1)

#Convert Pressure in inches of mercury to mbar
data_cleaned["Pressure(mbar)"] = data_cleaned["Pressure(in)"]*0.033863886666667*1000
data_cleaned.drop(['Pressure(in)'], inplace = True, axis = 1)

#Convert Wind Speed in mph to kmh
data_cleaned["Wind_Speed(kmh)"] = data_cleaned["Wind_Speed(mph)"]*1.609344
data_cleaned.drop(['Wind_Speed(mph)'], inplace = True, axis = 1)

#Convert Precipitation in inches to mm
data_cleaned["Precipitation(mm)"] = data_cleaned["Precipitation(in)"]*25.4
data_cleaned.drop(['Precipitation(in)'], inplace = True, axis = 1)

#Convert Visibility in miles to km
data_cleaned["Visibility(km)"] = data_cleaned["Visibility(mi)"]*1.609344
data_cleaned.drop(['Visibility(mi)'], inplace = True, axis = 1)
data_cleaned.head()



data_cleaned = data_cleaned[data_cleaned['Severity'] >= 1]
data_cleaned = data_cleaned.sort_values(by=['Severity'], ascending = True)
data_cleaned.reset_index()
data_cleaned.head()

t_col = 'Temperature(\u00b0C)'
h_col = 'Humidity(%)'

In [ ]:
data_cleaned.describe()

In [ ]:
fig = go.Figure(data=[go.Histogram(x=(data_cleaned['Severity']))])

fig.show()

#Frequency of severities
Severity_frequency = {}

for i, u in enumerate(range(0,5)):
    Severity_frequency['Severity' + str(u)] = (data_cleaned[(data_cleaned['Severity']==i)]).shape[0]

print(Severity_frequency)


In [ ]:
#Histogram Temperature
hist_temperature = go.Figure(data=[go.Histogram(x=(data_cleaned[t_col]), nbinsx=30)])

hist_temperature.update_layout(height = 450, width = 800,
    xaxis_title="Temperature (\u00b0C)",
    yaxis_title="Frequency"
)

hist_temperature.show()



#Lowest/Highest Temp
print("Lowest temperature in dataset is " + str((data_cleaned[t_col]).min()) + " \u00b0C")

print("Highest temperature in dataset is " + str((data_cleaned[t_col]).max()) + " \u00b0C")



#Left and right member in the low temperature interval
t_low1 = -70
t_low2 = -55

temperature_values_low = data_cleaned[t_col].between(t_low1, t_low2, inclusive = True)
print("There are " + str((data_cleaned[temperature_values_low]).shape[0]) + 
      " temperature values between " + str(t_low1) + " and " + str(t_low2) + " \u00b0C")

#Left member in the interval
t_high1 = 55

#Right number in the interval
t_high2 = 80

temperature_values_high = data_cleaned[t_col].between(t_high1, t_high2, inclusive = True)

print("There are " + str((data_cleaned[temperature_values_high]).shape[0]) + 
      " temperature values between " + str(t_high1) + " and " + str(t_high2) + " \u00b0C")


#Outliers (extremes) removal
data_cleaned_bool = data_cleaned[t_col].between(-55, 55, inclusive = True)
data_cleaned_temp = data_cleaned[data_cleaned_bool]
print(str(((data_cleaned[data_cleaned[t_col] > 55]).shape[0])+((data_cleaned[data_cleaned[t_col] < -55]).shape[0])) + 
      " measurements have been removed from the dataset ")
data_cleaned.head()

#Temperature Between Severity Groups Histograms

#Severity 1

sev1_temp = ((data_cleaned_temp[(data_cleaned_temp['Severity']) == 1])[t_col])
sev2_temp = ((data_cleaned_temp[(data_cleaned_temp['Severity']) == 2])[t_col])
sev3_temp = ((data_cleaned_temp[(data_cleaned_temp['Severity']) == 3])[t_col])
sev4_temp = ((data_cleaned_temp[(data_cleaned_temp['Severity']) == 4])[t_col])


fig = make_subplots(rows=2, cols=2, subplot_titles=("Severity 1","Severity 2", "Severity 3", "Severity 4"))

#Adding Traces
fig.add_trace(go.Histogram(x=(sev1_temp), nbinsx=20), row=1, col=1)
fig.add_trace(go.Histogram(x=(sev2_temp), nbinsx=20), row=1, col=2)
fig.add_trace(go.Histogram(x=(sev3_temp), nbinsx=20), row=2, col=1)
fig.add_trace(go.Histogram(x=(sev4_temp), nbinsx=20), row=2, col=2)

# Update xaxis properties
fig.update_xaxes(title_text="Temperature \u00b0C", row=1, col=1)
fig.update_xaxes(title_text="Temperature \u00b0C", row=1, col=2)
fig.update_xaxes(title_text="Temperature \u00b0C", row=2, col=1)
fig.update_xaxes(title_text="Temperature \u00b0C", row=2, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="Frequency", row=1, col=1)
fig.update_yaxes(title_text="Frequency", row=1, col=2)
fig.update_yaxes(title_text="Frequency", row=2, col=1)
fig.update_yaxes(title_text="Frequency", row=2, col=2)

# Update title and height
fig.update_layout(title_text="Variable Temperature \u00b0C histograms for each severity group", height=800)

fig.show()

In [ ]:
#If needed activate the code to remove outliers!

#Outliers (extremes) removal

column_name = input("Enter column name without brackets. Possible choices: " + str(list(data_cleaned.columns)) + " : ")

lower_value = int(input("Enter lowest included value in your chosen column for your new dataset "))
higher_value = int(input("Enter highest included value in your chosen column for your new dataset "))
data_cleaned_bool = data_cleaned[column_name].between(lower_value, higher_value, inclusive = True)
data_cleaned_new = data_cleaned[data_cleaned_bool]
print(str(((data_cleaned[data_cleaned[column_name] > higher_value]).shape[0])+
          ((data_cleaned[data_cleaned[column_name] < lower_value]).shape[0])) + 
      " measurements have been removed from the dataset \n")


print(data_cleaned_new.head())

print("\n Your new compiled dataset is available under the name of data_cleaned_new")

In [ ]:
#Left and right member in the low temperature interval
t_low1 = 85
t_low2 = 1000

temperature_values_low = data_cleaned_new['Visibility(km)'].between(t_low1, t_low2, inclusive = True)
print("There are " + str((data_cleaned_new[temperature_values_low]).shape[0]) + 
      " temperature values between " + str(t_low1) + " and " + str(t_low2) + " \u00b0C")

In [ ]:
#Histogram Universal Code

#Select dataset you want to work with (either data_cleaned or data_cleaned_new after removal of outliers)
data_input = data_cleaned_new

column_name = input("Enter column name without brackets. Possible choices: " + str(list(data_cleaned.columns)) + " : ")
hist_bins = int(input("Enter number of bins in histogram "))

hist_variable = go.Figure(data=[go.Histogram(x=(data_input[column_name]), nbinsx=hist_bins)])

hist_variable.update_layout(height = 450, width = 800,
    xaxis_title=column_name,
    yaxis_title="Frequency"
)

hist_variable.show()


#Lowest/Highest Value
print("Lowest " + column_name + " in dataset is " + str((data_input[column_name]).min()))

print("Highest " + column_name + " in dataset is " + str((data_input[column_name]).max()))



#Histograms Between Severity Groups

sev1 = ((data_input[(data_input['Severity']) == 1])[column_name])
sev2 = ((data_input[(data_input['Severity']) == 2])[column_name])
sev3 = ((data_input[(data_input['Severity']) == 3])[column_name])
sev4 = ((data_input[(data_input['Severity']) == 4])[column_name])






fig = make_subplots(rows=2, cols=2, subplot_titles=("Severity 1","Severity 2", "Severity 3", "Severity 4"))

#Adding Traces
fig.add_trace(go.Histogram(x=(sev1), nbinsx=hist_bins), row=1, col=1)
fig.add_trace(go.Histogram(x=(sev2), nbinsx=hist_bins), row=1, col=2)
fig.add_trace(go.Histogram(x=(sev3), nbinsx=hist_bins), row=2, col=1)
fig.add_trace(go.Histogram(x=(sev4), nbinsx=hist_bins), row=2, col=2)

# Update xaxis properties
fig.update_xaxes(title_text=column_name, row=1, col=1)
fig.update_xaxes(title_text=column_name, row=1, col=2)
fig.update_xaxes(title_text=column_name, row=2, col=1)
fig.update_xaxes(title_text=column_name, row=2, col=2)

# Update yaxis properties
fig.update_yaxes(title_text="Frequency", row=1, col=1)
fig.update_yaxes(title_text="Frequency", row=1, col=2)
fig.update_yaxes(title_text="Frequency", row=2, col=1)
fig.update_yaxes(title_text="Frequency", row=2, col=2)

# Update title and height
fig.update_layout(title_text="Variable " + column_name + " histograms for each Severity group", height=800)

fig.show()

In [ ]:
#Box Plot Temperature
box_plot_temperature = px.box(data_cleaned_temp, x="Severity", y=t_col)
box_plot_temperature.show()
Humidity (%)


In [ ]:
#Anderson-Darling Test Temperature

from scipy.stats import normaltest
sev1_temp = ((data_cleaned_temp[(data_cleaned_temp['Severity']) == 1])[t_col])
sev1_temp_np = sev1_temp.to_numpy()
sev1_temp_np = sev1_temp_np[np.logical_not(np.isnan(sev1_temp_np))]

from scipy.stats import anderson

#1
result = anderson(sev1_temp_np)
stat = round(result.statistic, 4)

p = 0
result_mat = []
for i in range(len(result.critical_values)):
    title = "Severity " + str(int((i+3)%(i+2)))
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < result.critical_values[i]:
        msg = 'Sample looks Gaussian (fail to reject H0)'
    else:
        msg = 'Sample does not look Gaussian (reject H0)'
    result_mat.append([title, len(sev1_temp_np), stat, sl, cv, msg])

trace = go.Table(
    header=dict(values=['<b>Title<b>', '<b>Sample Size</b>', '<b>Statistic</b>', '<b>Significance Level</b>', '<b>Critical Value</b>', '<b>Comment</b>'],
                line = dict(width=0),
                fill = dict(color='rgba(42,63,95,0.8)'),
                align = 'center',
                font = dict(
                    color = '#ffffff',
                    size = 12
                )),
    cells=dict(values=np.array(result_mat).T,
               line = dict(width=0),
               fill = dict(color=[['#EBF0F8', '#ffffff', '#EBF0F8', '#ffffff', '#EBF0F8']]),
               align = 'center',
               height = 40),
    columnwidth=[0.3, 0.25, 0.3, 0.25, 0.5])
layout = dict(
    height=300,
    margin=dict(
        l=5,
        r=5,
        t=30,
        b=0
    )
)
data = [trace]
andar_table = dict(data=data, layout=layout)

iplot(andar_table, filename='anderson-darling-table')

#2

sev2_temp = ((data_cleaned_temp[(data_cleaned_temp['Severity']) == 2])[t_col])
sev2_temp_np = sev2_temp.to_numpy()
sev2_temp_np = sev2_temp_np[np.logical_not(np.isnan(sev2_temp_np))]

result = anderson(sev2_temp_np)
stat = round(result.statistic, 4)

p = 0
result_mat = []
for i in range(len(result.critical_values)):
    title = "Severity " + str(int((i+6)%(i+4)))
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < result.critical_values[i]:
        msg = 'Sample looks Gaussian (fail to reject H0)'
    else:
        msg = 'Sample does not look Gaussian (reject H0)'
    result_mat.append([title, len(sev2_temp_np), stat, sl, cv, msg])

trace = go.Table(
    header=dict(values=['<b>Title<b>', '<b>Sample Size</b>', '<b>Statistic</b>', '<b>Significance Level</b>', '<b>Critical Value</b>', '<b>Comment</b>'],
                line = dict(width=0),
                fill = dict(color='rgba(42,63,95,0.8)'),
                align = 'center',
                font = dict(
                    color = '#ffffff',
                    size = 12
                )),
    cells=dict(values=np.array(result_mat).T,
               line = dict(width=0),
               fill = dict(color=[['#EBF0F8', '#ffffff', '#EBF0F8', '#ffffff', '#EBF0F8']]),
               align = 'center',
               height = 40),
    columnwidth=[0.3, 0.25, 0.3, 0.25, 0.5])
layout = dict(
    height=300,
    margin=dict(
        l=5,
        r=5,
        t=30,
        b=0
    )
)
data = [trace]
andar_table = dict(data=data, layout=layout)

iplot(andar_table, filename='anderson-darling-table')

#3

sev3_temp = ((data_cleaned_temp[(data_cleaned_temp['Severity']) == 3])[t_col])
sev3_temp_np = sev3_temp.to_numpy()
sev3_temp_np = sev3_temp_np[np.logical_not(np.isnan(sev3_temp_np))]

result = anderson(sev3_temp_np)
stat = round(result.statistic, 4)

p = 0
result_mat = []
for i in range(len(result.critical_values)):
    title = "Severity " + str(int((i+7)%(i+4)))
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < result.critical_values[i]:
        msg = 'Sample looks Gaussian (fail to reject H0)'
    else:
        msg = 'Sample does not look Gaussian (reject H0)'
    result_mat.append([title, len(sev3_temp_np), stat, sl, cv, msg])

trace = go.Table(
    header=dict(values=['<b>Title<b>', '<b>Sample Size</b>', '<b>Statistic</b>', '<b>Significance Level</b>', '<b>Critical Value</b>', '<b>Comment</b>'],
                line = dict(width=0),
                fill = dict(color='rgba(42,63,95,0.8)'),
                align = 'center',
                font = dict(
                    color = '#ffffff',
                    size = 12
                )),
    cells=dict(values=np.array(result_mat).T,
               line = dict(width=0),
               fill = dict(color=[['#EBF0F8', '#ffffff', '#EBF0F8', '#ffffff', '#EBF0F8']]),
               align = 'center',
               height = 40),
    columnwidth=[0.3, 0.25, 0.3, 0.25, 0.5])
layout = dict(
    height=300,
    margin=dict(
        l=5,
        r=5,
        t=30,
        b=0
    )
)
data = [trace]
andar_table = dict(data=data, layout=layout)

iplot(andar_table, filename='anderson-darling-table')

#4

sev4_temp = ((data_cleaned_temp[(data_cleaned_temp['Severity']) == 4])[t_col])
sev4_temp_np = sev4_temp.to_numpy()
sev4_temp_np = sev4_temp_np[np.logical_not(np.isnan(sev4_temp_np))]

result = anderson(sev4_temp_np)
stat = round(result.statistic, 4)

p = 0
result_mat = []
for i in range(len(result.critical_values)):
    title = "Severity " + str(int((i+9)%(i+5)))
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < result.critical_values[i]:
        msg = 'Sample looks Gaussian (fail to reject H0)'
    else:
        msg = 'Sample does not look Gaussian (reject H0)'
    result_mat.append([title, len(sev4_temp_np), stat, sl, cv, msg])

trace = go.Table(
    header=dict(values=['<b>Title<b>', '<b>Sample Size</b>', '<b>Statistic</b>', '<b>Significance Level</b>', '<b>Critical Value</b>', '<b>Comment</b>'],
                line = dict(width=0),
                fill = dict(color='rgba(42,63,95,0.8)'),
                align = 'center',
                font = dict(
                    color = '#ffffff',
                    size = 12
                )),
    cells=dict(values=np.array(result_mat).T,
               line = dict(width=0),
               fill = dict(color=[['#EBF0F8', '#ffffff', '#EBF0F8', '#ffffff', '#EBF0F8']]),
               align = 'center',
               height = 40),
    columnwidth=[0.3, 0.25, 0.3, 0.25, 0.5])
layout = dict(
    height=300,
    margin=dict(
        l=5,
        r=5,
        t=30,
        b=0
    )
)
data = [trace]
andar_table = dict(data=data, layout=layout)

iplot(andar_table, filename='anderson-darling-table')

In [ ]:

from scipy.stats import shapiro

# normality test
stat, p = shapiro(Severity3)
print('Statistics=%.3f, p=%.3f' % (stat, p))
# interpret
alpha = 0.05
if p > alpha:
	print('Sample looks Gaussian (fail to reject H0)')
else:
	print('Sample does not look Gaussian (reject H0)')

In [ ]:
#Anderson-Darling Test Universal

from scipy.stats import normaltest
from scipy.stats import anderson

#Choose dataset with what to work (data_cleaned or data_cleaned_new)
data_input = data_cleaned_new

column_name = input("Enter column name without brackets. Possible choices: " + 
                    str(list(data_cleaned.columns)) + " : ")

sev1_and = ((data_input[(data_input['Severity']) == 1])[column_name])
Severity1 = sev1_and.to_numpy()
Severity1 = Severity1[np.logical_not(np.isnan(Severity1))]

sev2_and = ((data_input[(data_input['Severity']) == 2])[column_name])
Severity2 = sev2_and.to_numpy()
Severity2 = Severity2[np.logical_not(np.isnan(Severity2))]

sev3_and = ((data_input[(data_input['Severity']) == 3])[column_name])
Severity3 = sev3_and.to_numpy()
Severity3 = Severity3[np.logical_not(np.isnan(Severity3))]

sev4_and = ((data_input[(data_input['Severity']) == 4])[column_name])
Severity4 = sev4_and.to_numpy()
Severity4 = Severity4[np.logical_not(np.isnan(Severity4))]

sev_list = [Severity1, Severity2, Severity3, Severity4]

counter = 0

for i in sev_list:
    result = anderson(i)
    print("\033[1m" + (namestr(i, globals())[0] + ' Statistic: %.3f \033[0m' % result.statistic))
    while counter <1:
        p = 0
        for i in range(len(result.critical_values)):
            sl, cv = result.significance_level[i], result.critical_values[i]
            if result.statistic < result.critical_values[i]:
                print('Significance level: %.3f: Critical Value : %.3f, data looks normal (fail to reject H0)' % (sl, cv))
            else:
                print('Significance level: %.3f: Critical Value : %.3f, data does not look normal (reject H0)' % (sl, cv))
            counter += 1
    counter = 0

In [ ]:
from scipy.stats import ttest_ind

Severity1 = np.copy(sev1_temp_np)
Severity2 = np.copy(sev2_temp_np)
Severity3 = np.copy(sev3_temp_np)
Severity4 = np.copy(sev4_temp_np)

sev_all_temp = [Severity1, Severity2, Severity3, Severity4]
sev_all_temp2 = [Severity2, Severity3, Severity4]

statistics={}
p_values={}

    
counter = 0

for i in sev_all_temp:
    while sev_all_temp2 and counter <1:
        for u in sev_all_temp2:
            (t,p) = ttest_ind(i, u)
            if p == 1 or t == 0:
                pass
            else:
                p_values[(namestr(i, globals())[1] + " vs " + namestr(u, globals())[1])]=p
                statistics[(namestr(i, globals())[1] + " vs " + namestr(u, globals())[1])]=t
                counter += 1
    counter = 0
    

t_test_pd1 = pd.DataFrame.from_dict(statistics, orient='index')
t_test_pd1.columns = ['Statistics']

t_test_pd2 = pd.DataFrame.from_dict(p_values, orient='index')
t_test_pd2.columns = ['P Value']

result = t_test_pd1.join(t_test_pd2)

result = result.drop([result.index[5], result.index[7], result.index[8]])
print(result)

In [ ]:
##Function
#
#list = ["a", "b", "c", "d"]
#list2 = ["b", "c", "d"]
#
#counter = 0
#
#for i in list:
#    while counter <1:
#        for u in list2:
#            if i == u:
#                pass
#            else:
#                print(i+u) 
#            counter += 1
#    counter = 0

In [ ]:
from statsmodels.stats import weightstats as stests

z_statistics={}
z_p_values={}

    
counter = 0

for i in sev_all_temp:
    while sev_all_temp2 and counter <1:
        for u in sev_all_temp2:
            ztest, pval = stests.ztest(i, u, value=0, alternative='two-sided')
            if pval == 1 or ztest == 0:
                pass
            else:
                z_p_values[namestr(i, globals())[1]+ " vs " + namestr(u, globals())[1]]=pval
                z_statistics[namestr(i, globals())[1]+ " vs " + namestr(u, globals())[1]]=ztest
                counter += 1
    counter = 0
    

z_test_pd1 = pd.DataFrame.from_dict(z_statistics, orient='index')
z_test_pd1.columns = ['Z Statistics']

z_test_pd2 = pd.DataFrame.from_dict(z_p_values, orient='index')
z_test_pd2.columns = ['Z P Value']

result_z = z_test_pd1.join(z_test_pd2)

result_z = result_z.drop([result_z.index[5], result_z.index[7], result_z.index[8]])
print(result_z)

In [ ]:
import scipy.stats as stats

#ANOVA with outliers

F, p = stats.f_oneway(sev1_temp, sev2_temp, sev3_temp, sev4_temp)
print("ANOVA with outliers - p-value : ", p)
print("ANOVA with outliers - F- value:", F)
    
    
#ANOVA after removing outliers

data_new = data_cleaned[data_cleaned[t_col].between(-30, 30, inclusive=True)]
data_new = data_new.dropna()
sev1_new = data_new[data_new['Severity'] == 1]
sev1_new = sev1_new[t_col]
sev2_new = data_new[data_new['Severity'] == 2]
sev2_new = sev2_new[t_col]
sev3_new = data_new[data_new['Severity'] == 3]
sev3_new = sev3_new[t_col]
sev4_new = data_new[data_new['Severity'] == 4]
sev4_new = sev4_new[t_col]


F, p = stats.f_oneway(sev1_new, sev2_new, sev3_new, sev4_new)
print("ANOVA removed outliers - p-value : ", p)
print("ANOVA removed outliers - F- value:", F)




    

    
    

In [ ]:
#Kruskal-Wallis Test

from scipy.stats import kruskal

stat, p = kruskal(sev1_temp, sev2_temp, sev3_temp, sev4_temp)
print("Kruskal-Wallis with outliers - p-value : ", p)
print("Kruskal-Wallis with outliers - Statistics:", stat)

stat, p = kruskal(sev1_new, sev2_new, sev3_new, sev4_new)
print("Kruskal-Wallis removed outliers - p-value : ", p)
print("Kruskal-Wallis removed outliers - Statistics:", stat)


In [ ]:
data_sev_temp = data_cleaned_temp[['Severity', t_col]]
                          
data_sev_temp = data_sev_temp.dropna()

data_sev_temp = data_sev_temp.sort_values(by=['Severity'])

data_sev_temp2 = data_sev_temp.drop(columns='Severity')

data_sev_temp = (data_sev_temp['Severity']).to_numpy()

#Ordinal Logistic Regression

from bevel.linear_ordinal_regression import OrderedLogit

ls = OrderedLogit()
ls.fit(data_sev_temp2, data_sev_temp)

ls.print_summary()